In [15]:
# Version 4.2
# CHANGE: border mode - changed from valid to same
# Conclusion - results are very similar for both settings, sometimes a 1-2 % better with "same"

In [16]:
#all imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
#import cv2
import csv
import imageio

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras import backend as K


In [17]:
K.set_image_dim_ordering('th')
seed = 7
np.random.seed(seed)

In [18]:
def rgb2gray(rgb):

    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

def readimagesfromfolder(path, color, show):
	imagepackage_list = []
	imagepackage = []
	imagelist = []
	filelist = sorted(os.listdir(path))
	for filename in filelist:
		fullfilename =  path + filename
		#print(fullfilename)        
		singleimage = imageio.imread(fullfilename)
		singleimage = rgb2gray(singleimage)
		#print(singleimage)        
		if show==1:
			plt.imshow(singleimage)
			plt.show()
		imagepackage.append(singleimage)
		imagepackage_list.append(imagepackage)        
		imagepackage = []        
	#print(len(imagepackage))        
	imagepackage_list = np.float32(imagepackage_list)
	#cv2.destroyAllWindows()
	return imagepackage_list

def readlabelfromcsv(filename):
    training_y = []
    with open(filename) as f:
        #training_y.append(f.readline())
        content = csv.reader(f)
        for line in content:
            #print(line[0])
            training_y.append(int(line[0]))
    return training_y




In [19]:
       
folderpath = 'data/basic/training/'
X_train=readimagesfromfolder(folderpath, 0, 0)
print(np.shape(X_train))

folderpath = 'data/basic/test/'
X_test=readimagesfromfolder(folderpath, 0, 0)
print(np.shape(X_test))

y_train = readlabelfromcsv("data/basic/training.csv")
y_test = readlabelfromcsv("data/basic/test.csv")

print(y_train)
print(np.shape(y_train))

print(y_test)
print(np.shape(y_test))

datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=4,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False) # randomly flip images

datagen.fit(X_train)

(162, 1, 180, 180)
(48, 1, 180, 180)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
(162,)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
(48,)


In [20]:


# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [21]:
def larger_model():
	# create model
	model = Sequential()
	model.add(Convolution2D(24, 5, 5, border_mode='same', input_shape=(1, 180, 180), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Convolution2D(16, 3, 3, border_mode='same', activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.5))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dense(50, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

# build the model
model = larger_model()
# Fit the model
model.fit_generator(datagen.flow(X_train, y_train, batch_size=16),samples_per_epoch = 5000, nb_epoch = 20, verbose=2, validation_data=(X_test, y_test))
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print scores
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

Epoch 1/20
24s - loss: 0.6175 - acc: 0.7438 - val_loss: 0.5210 - val_acc: 0.7292
Epoch 2/20
24s - loss: 0.3360 - acc: 0.8775 - val_loss: 0.5426 - val_acc: 0.7500
Epoch 3/20
24s - loss: 0.2546 - acc: 0.9069 - val_loss: 0.6140 - val_acc: 0.7708
Epoch 4/20
24s - loss: 0.1283 - acc: 0.9553 - val_loss: 0.5513 - val_acc: 0.8333
Epoch 5/20
24s - loss: 0.1020 - acc: 0.9660 - val_loss: 0.8190 - val_acc: 0.7708
Epoch 6/20
24s - loss: 0.0837 - acc: 0.9724 - val_loss: 0.8769 - val_acc: 0.7500
Epoch 7/20
24s - loss: 0.0337 - acc: 0.9880 - val_loss: 1.0607 - val_acc: 0.8333
Epoch 8/20
24s - loss: 0.0343 - acc: 0.9888 - val_loss: 1.0831 - val_acc: 0.8542
Epoch 9/20
25s - loss: 0.0583 - acc: 0.9808 - val_loss: 0.8809 - val_acc: 0.7708
Epoch 10/20
24s - loss: 0.0156 - acc: 0.9948 - val_loss: 1.0581 - val_acc: 0.8542
Epoch 11/20
24s - loss: 0.0505 - acc: 0.9846 - val_loss: 1.1990 - val_acc: 0.7500
Epoch 12/20
24s - loss: 0.0158 - acc: 0.9942 - val_loss: 0.9359 - val_acc: 0.8125
Epoch 13/20
24s - loss: 0